In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 23.6 MB/s 
     |████████████████████████████████| 163 kB 58.2 MB/s 
     |████████████████████████████████| 7.6 MB 47.6 MB/s 


In [ ]:
import warnings
import pandas as pd
import numpy as np
from tqdm import tqdm

#import tensorflow as tf
import torch
from keras_preprocessing.sequence import pad_sequences
from transformers import BertTokenizer,  AutoModelForSequenceClassification

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

data  = pd.read_csv("/content/drive/MyDrive/AI5 MLOPS/papers_with_abstract.csv")
print("Data Shape :" , data.shape)

Data Shape : (9313, 6)


Data Preprocessing 

In [ ]:
data.head()

,Unnamed: 0,source_id,year,title,abstract,full_text
0,0,27,1987,Bit-Serial Neural Networks,\n\nA bit - serial VLSI neural network i...,573 \n\nBIT - SERIAL NEURAL NETWORKS \n\nAlan...
1,1,63,1987,Connectivity Versus Entropy,\n\nHow does the connectivity of a neural...,1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser S...
2,2,60,1987,The Hopfield Model with Multi-Level Neurons,\n\nThe Hopfield neural network. model fo...,278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...
3,3,59,1987,How Neural Nets Work,\n\nThere is presently great interest in the...,442 \n\nAlan Lapedes \nRobert Farber \n\nThe...
4,4,69,1987,Spatial Organization of Neural Networks: A Pro...,\n\nThe aim of this paper is to explore...,740 \n\nSPATIAL ORGANIZATION OF NEURAL NEn...


In [ ]:
data = data.drop(["Unnamed: 0","source_id"],axis = 1)
data['abstract'] = data['abstract'].replace(r'\n', ' ', regex=True) 
data['full_text'] = data['full_text'].replace(r'\n', ' ', regex=True) 

In [ ]:
data.loc[0]["abstract"]

'  A  bit  - serial  VLSI  neural  network  is  described  from  an  initial  architecture  for  a  synapse array through to silicon layout and board design.  The issues surrounding bit  - serial  computation,  and  analog/digital  arithmetic  are  discussed  and  the  parallel  development  of  a  hybrid  analog/digital  neural  network  is  outlined.  Learning  and  recall  capabilities  are  reported  for  the  bit  - serial  network  along  with  a  projected  specification  for  a  64  - neuron,  bit  - serial  board  operating  at 20 MHz.  This tech(cid:173) nique  is  extended  to  a  256  (2562  synapses)  network  with  an  update  time  of 3ms,  using  a  "paging"  technique  to  time  - multiplex  calculations  through  the  synapse  array.   1. '

Vectorization

In [ ]:

# Get the SciBERT pretrained model path from Allen AI repo
pretrained_model = 'allenai/scibert_scivocab_uncased'

# Get the tokenizer from the previous path
sciBERT_tokenizer = BertTokenizer.from_pretrained(pretrained_model, 
                                          do_lower_case=True)

# Get the model
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model,
                                                          output_attentions=False,
                                                          output_hidden_states=True)

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [ ]:
def convert_single_abstract_to_embedding(tokenizer, model, in_text, MAX_LEN = 510):
    
    input_ids = tokenizer.encode(
                        in_text, 
                        add_special_tokens = True, 
                        max_length = MAX_LEN,                           
                   )    
    
    print("input ids",input_ids)

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    print("results",results)
    
    # Remove the outer list.
    input_ids = results[0]
    #print("input ids",input_ids)

    # Create attention masks    
    attention_mask = [int(i>0) for i in input_ids]
    #print("attention_mask",attention_mask)
    
    # Convert to tensors.
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    #print("input ids",input_ids)
    #print("attention_mask",attention_mask)

    # Add an extra dimension for the "batch" (even though there is only one 
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    #print("input ids",input_ids)
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    #input_ids = input_ids.to(device)
    #attention_mask = attention_mask.to(device)
    
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():        
        logits, encoded_layers = model(
                                    input_ids = input_ids, 
                                    token_type_ids = None, 
                                    attention_mask = attention_mask,
                                    return_dict=False)

    layer_i = 12 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]
        
    # Extract the embedding.
    embedding = encoded_layers[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    embedding = embedding.detach().cpu().numpy()

    return(embedding)

In [ ]:
input_abstract = data.abstract.iloc[30]
print(input_abstract)

# Use the model and tokenizer to generate an embedding for the input_abstract
abstract_embedding = convert_single_abstract_to_embedding(sciBERT_tokenizer, model, input_abstract)

print('Embedding shape: {}'.format(abstract_embedding.shape))
abstract_embedding.shape

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  We  have  studied  the  basins  of  attraction  for  fixed  point  and  oscillatory  attractors  in  an  electronic  analog  neural  network.  Basin  measurement  circuitry  periodically  opens  the  network  feedback  loop,  loads  raster-scanned  initial  conditions  and  examines  the  resulting  attractor.  Plotting  the  basins  for  fixed  points  (memories),  we  show  that  overloading  an  associative  memory  network  leads  to  irregular  basin  shapes.  The  network  also  includes  analog  time  delay  circuitry,  and  we  have  shown  that  delay  in  symmetric  networks  can  introduce  basins  for  oscillatory  attractors.  Conditions  leading  to  oscillation  are  related  to  the  presence  of  frustration;  reducing  frustration  by  diluting  the  connections  can  stabilize  a  delay  network.   (1)  -  
input ids [102, 185, 360, 2580, 111, 25551, 131, 18158, 168, 2612, 1313, 137, 20043, 26738, 30113, 121, 130, 4548, 3936, 3687, 934, 205, 12017, 2560, 20882, 165

(768,)

In [ ]:

def convert_overall_text_to_embedding(df):
    
    # The list of all the embeddings
    embeddings = []
    
    # Get overall text data
    overall_text_data = data.iloc[:2000]["abstract"].values
    
    # Loop over all the comment and get the embeddings
    for abstract in tqdm(overall_text_data):
        
        # Get the embedding 
        embedding = convert_single_abstract_to_embedding(sciBERT_tokenizer, model, abstract)
        
        #add it to the list
        embeddings.append(embedding)
        
    print("Conversion Done!")
    
    return embeddings

In [ ]:
embeddings = convert_overall_text_to_embedding(data)

100%|██████████| 2000/2000 [37:20<00:00,  1.12s/it]

Conversion Done!


In [ ]:
def create_final_embeddings(df, embeddings):
    
    df["embeddings"] = embeddings
    df["embeddings"] = df["embeddings"].apply(lambda emb: np.array(emb))
    df["embeddings"] = df["embeddings"].apply(lambda emb: emb.reshape(1, -1))
    
    return df
data_2000 = create_final_embeddings(data.head(2000), embeddings)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,year,title,abstract,full_text,embeddings
0,1987,Bit-Serial Neural Networks,A bit - serial VLSI neural network is ...,573 BIT - SERIAL NEURAL NETWORKS Alan F. ...,"[[-0.46600834, 0.54790574, -0.74426436, -0.127..."
1,1987,Connectivity Versus Entropy,How does the connectivity of a neural ...,1 CONNECTIVITY VERSUS ENTROPY Yaser S. A...,"[[0.13005306, -0.11771801, 0.18673722, 0.25362..."
2,1987,The Hopfield Model with Multi-Level Neurons,The Hopfield neural network. model for ...,278 THE HOPFIELD MODEL WITH MUL TI-LEVEL NEU...,"[[0.034975298, 0.20073466, -0.7868019, -0.4869..."


In [ ]:
data_2000.head(10)

,year,title,abstract,full_text,embeddings
0,1987,Bit-Serial Neural Networks,A bit - serial VLSI neural network is ...,573 BIT - SERIAL NEURAL NETWORKS Alan F. ...,"[[-0.46600834, 0.54790574, -0.74426436, -0.127..."
1,1987,Connectivity Versus Entropy,How does the connectivity of a neural ...,1 CONNECTIVITY VERSUS ENTROPY Yaser S. A...,"[[0.13005306, -0.11771801, 0.18673722, 0.25362..."
2,1987,The Hopfield Model with Multi-Level Neurons,The Hopfield neural network. model for ...,278 THE HOPFIELD MODEL WITH MUL TI-LEVEL NEU...,"[[0.034975298, 0.20073466, -0.7868019, -0.4869..."
3,1987,How Neural Nets Work,There is presently great interest in the a...,442 Alan Lapedes Robert Farber Theoreti...,"[[0.02412813, -0.37279242, -0.18595831, -0.260..."
4,1987,Spatial Organization of Neural Networks: A Pro...,The aim of this paper is to explore ...,740 SPATIAL ORGANIZATION OF NEURAL NEn~O...,"[[0.7227678, -1.0552447, -0.2167558, 0.2328343..."
5,1987,A Neural-Network Solution to the Concentrator ...,Networks of simple analog processors havi...,775 A NEURAL-NETWORK SOLUTION TO THE CON...,"[[0.5478606, -0.67029506, 0.27171218, -0.21017..."
6,1987,LEARNING BY STATE RECURRENCE DETECTION,his research investigates a new technique for ...,642 LEARNING BY ST ATE RECURRENCE DETECfION ...,"[[-0.14074916, -1.0156024, 0.88289535, 0.49035..."
7,1987,Stability Results for Neural Networks,In the present paper we survey and utilize r...,554 STABILITY RESULTS FOR NEURAL NETWORKS ...,"[[0.177729, -0.8746792, 0.27143663, -0.4612798..."
8,1987,Introduction to a System for Implementing Neur...,Neural networks have attracted much int...,804 INTRODUCTION TO A SYSTEM FOR IMPLEMENTI...,"[[0.25519475, -0.93505615, 0.6953258, -0.63813..."
9,1987,Optimization with Artificial Neural Network Sy...,General formulae for mapping optimizatio...,474 OPTIMIZA nON WITH ARTIFICIAL NEURAL NETW...,"[[0.07592578, -0.42082918, 0.75627637, -0.5823..."


In [ ]:
# Get the query text
query_text_test = "HUMAN BRAIN MIMICS"
# Get the similar articles
top_articles = get_top_N_articles_cosine(query_text_test, data_2000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
top_articles

,title,abstract,cos_sim
38,Simulations Suggest Information Processing Rol...,computer model of the hippocampal pyramidal c...,0.755635
99,GENESIS: A System for Simulating Neural Networks,support simulations at many it is We ...,0.732277
37,SPONTANEOUS AND INFORMATION-TRIGGERED SEGMENT...,he brain works in a state-dependent manner: pr...,0.719951
96,Automatic Local Annealing,This research involves a method for finding ...,0.712556
400,Visual Grammars and Their Neural Nets,exhibit a systematic way to derive neu...,0.710362


In [ ]:
print("The Query : ",query_text_test)
for n,i in enumerate(top_articles.title.values):
  print(f"{n} : {i}")

The Query :  HUMAN BRAIN MIMICS
0 : SPONTANEOUS AND  INFORMATION-TRIGGERED SEGMENTS OF SERIES OF HUMAN BRAIN ELECTRIC FIELD MAPS
1 : HIGH DENSITY ASSOCIATIVE MEMORIES
2 : Neural Networks that Learn to Discriminate Similar Kanji Characters
3 : A Model of Feedback to the Lateral Geniculate Nucleus
4 : The Capacity of the Kanerva Associative Memory is Exponential
